<a href="https://colab.research.google.com/github/varunkumark1997/EVA5-Assignment/blob/master/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [68]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3) #input -? OUtput? RF
        self.bn1   = nn.BatchNorm2d(16)
        self.drop1 = nn.Dropout(0.25)
        self.conv2 = nn.Conv2d(16, 16, 3, stride=2)
        self.bn2   = nn.BatchNorm2d(16)
        self.drop2 = nn.Dropout(0.25)
        self.conv3 = nn.Conv2d(16, 32, 3)
        self.bn3   = nn.BatchNorm2d(32)      
        self.drop3 = nn.Dropout(0.25)  
        self.conv4 = nn.Conv2d(32, 32, 3)
        self.bn4   = nn.BatchNorm2d(32)      
        self.drop4 = nn.Dropout(0.25)          
        self.conv5 = nn.Conv2d(32, 10, 3)
        self.avg1  = nn.AvgPool2d(5, 5)

    def forward(self, x):
        x = self.drop2(self.bn2(F.relu(self.conv2(self.drop1(self.bn1(F.relu(self.conv1(x))))))))
        x = self.drop4(self.bn4(F.relu(self.conv4(self.drop3(self.bn3(F.relu(self.conv3(x))))))))
        x = self.avg1(self.conv5(x))
        x = x.view(x.size(0), -1)
        return F.log_softmax(x)


In [69]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
           Dropout-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 16, 12, 12]           2,320
       BatchNorm2d-5           [-1, 16, 12, 12]              32
           Dropout-6           [-1, 16, 12, 12]               0
            Conv2d-7           [-1, 32, 10, 10]           4,640
       BatchNorm2d-8           [-1, 32, 10, 10]              64
           Dropout-9           [-1, 32, 10, 10]               0
           Conv2d-10             [-1, 32, 8, 8]           9,248
      BatchNorm2d-11             [-1, 32, 8, 8]              64
          Dropout-12             [-1, 32, 8, 8]               0
           Conv2d-13             [-1, 10, 6, 6]           2,890
        AvgPool2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [70]:


torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [71]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [72]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0593, Accuracy: 9793/10000 (97.9%)



  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0476, Accuracy: 9861/10000 (98.6%)



  0%|          | 0/1875 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0225, Accuracy: 9934/10000 (99.3%)



  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0193, Accuracy: 9943/10000 (99.4%)



loss=0.0008432903559878469 batch_id=1874: 100%|██████████| 1875/1875 [00:22<00:00, 83.53it/s]



Test set: Average loss: 0.0179, Accuracy: 9949/10000 (99.5%)

